In [1]:
%config Completer.use_jedi = False

In [17]:
import igraph as ig
import pandas as pd
import numpy as np

## Graph parameters

In [4]:
nodes_array = [10e1, 10e2, 10e3]
probabilities_array = [0.3, 0.5, 0.9, 1]
edges_array = [10e1, 10e2, 10e3]
network_numbers = 3

## Graph building

In [5]:
# A Python program to print all
# permutations using library function
from itertools import product

# Get all permutations of [1, 2, 3]
perm = product(*[nodes_array, probabilities_array])

graphs_dict = dict()
# Print the obtained permutations
for network_property in list(perm):
    graphs_list = list()
    
    for i in range(network_numbers):
        
        nodes = int(network_property[0])
    #     edges = int(i[1])
        probability = network_property[1]
#         print(nodes, "nodes with ", probability, "probability")
        graph = ig.Graph.Erdos_Renyi(n = nodes, p = probability)
        graphs_list.append(graph)
    
    key = str(nodes) + '_nodes__' + str(probability) + "_probability"
    graphs_dict[key] = graphs_list

## Can you observe the connectivity threshold?

## Extract the giant components and characterize them

## Does the network diameter follows the logarithimic variation?

In [23]:
perm = product(*[nodes_array, probabilities_array])
# Print the obtained permutations
for network_property in list(perm):
    nodes = int(network_property[0])
    probability = network_property[1]
    z = probability*nodes
    print("z:", z, "log(n):", np.log(nodes), "q: ", 1/nodes)

z: 30.0 log(n): 4.605170185988092 q:  0.01
z: 50.0 log(n): 4.605170185988092 q:  0.01
z: 90.0 log(n): 4.605170185988092 q:  0.01
z: 100 log(n): 4.605170185988092 q:  0.01
z: 300.0 log(n): 6.907755278982137 q:  0.001
z: 500.0 log(n): 6.907755278982137 q:  0.001
z: 900.0 log(n): 6.907755278982137 q:  0.001
z: 1000 log(n): 6.907755278982137 q:  0.001
z: 3000.0 log(n): 9.210340371976184 q:  0.0001
z: 5000.0 log(n): 9.210340371976184 q:  0.0001
z: 9000.0 log(n): 9.210340371976184 q:  0.0001
z: 10000 log(n): 9.210340371976184 q:  0.0001
